In [1]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("JavaScript TfldExample");
    var sc = new SparkContext(sparkConf);


In [2]:
    var SQLContext = require('eclairjs/sql/SQLContext');

    var sqlContext = new SQLContext(sc);
    var RowFactory = require('eclairjs/sql/RowFactory');
    var StructType = require('eclairjs/sql/types/StructType');
    var StructField = require('eclairjs/sql/types/StructField');
    var DataTypes = require('eclairjs/sql/types').DataTypes;
    var Metadata = require('eclairjs/sql/types/Metadata');
    var Tokenizer = require('eclairjs/ml/feature/Tokenizer');
    var HashingTF = require('eclairjs/ml/feature/HashingTF');
    var IDF = require('eclairjs/ml/feature/IDF');


In [3]:

    var jrdd = sc.parallelize([
      RowFactory.create(0, "Hi I heard about Spark"),
      RowFactory.create(0, "I wish Java could use case classes"),
      RowFactory.create(1, "Logistic regression models are neat")
    ]);
    var schema = new StructType([
      new StructField("label", DataTypes.DoubleType, false, Metadata.empty()),
      new StructField("sentence", DataTypes.StringType, false, Metadata.empty())
    ]);

    var sentenceData = sqlContext.createDataFrame(jrdd, schema);
    var tokenizer = new Tokenizer().setInputCol("sentence").setOutputCol("words");
    var wordsData = tokenizer.transform(sentenceData);
    var numFeatures = 20;
    var hashingTF = new HashingTF()
      .setInputCol("words")
      .setOutputCol("rawFeatures")
      .setNumFeatures(numFeatures);
    var featurizedData = hashingTF.transform(wordsData);
    var idf = new IDF().setInputCol("rawFeatures").setOutputCol("features");
    var idfModel = idf.fit(featurizedData);
    var rescaledData = idfModel.transform(featurizedData);
    var rows=rescaledData.select("features", "label").take(3);

var result = rows;

    for (var i=0; i<result.length; i++) {
        var r=result[i];
      var features = r.get(0);
      var label = r.getDouble(1);
      print(features);
      print(label);
    }


 $example off$


In [4]:
    sc.stop();
